# Analyze Model
Below, this file is aimed at testing and analyzing the expansion model. It is not necessarily meant to be the robust testing script that iteratively runs the expansion model, but instead is meant to enable an intuition about the data, help with debugging, and assisst with formulating greater conclusions. 

In [25]:
using Gurobi
include("expansion_model.jl")

write_results (generic function with 2 methods)

In [2]:
# create input and output directory
inputs_path = joinpath(@__DIR__, "WECC_6zone_2035")
outputs_directory = joinpath(@__DIR__, "results/model_test_results")
if !(isdir(outputs_directory))
	mkdir(outputs_directory)
end

In [3]:
# load inputs
(generators, G) = load_input_generator(inputs_path)
(demand, nse, sample_weight, hours_per_period, P, S, W, T, Z) = load_input_demand(inputs_path)
variability = load_input_variability(inputs_path)
(lines, L) = load_input_network(inputs_path);

In [26]:
# create policy arrays
rps_values = [-1, -1, -1, -1, -1, -1]
ces_values = [-1, -1, -1, -1, -1, -1]
carbon_tax_values = [-1, -1, -1, -1, -1, -1]
cap_trade_clusters = []
cap_trade_values = []

# Basic ITC and PTC are already included in the function; but here we creat our own 
itc_values = [0, 0, 0, 0, 0, 0]
ptc_values = [0, 0, 0, 0, 0, 0]

6-element Vector{Int64}:
 0
 0
 0
 0
 0
 0

In [27]:
# Test model
(generator_results,
    storage_results,
    transmission_results,
    nse_results,
    no_policy_cost_results,
    time, no_policy_emissions_results, dual_results) = solve_cap_expansion(generators, G,
    demand, S, T, Z,
    nse,
    sample_weight, hours_per_period,
    lines, L,
    variability, Gurobi.Optimizer, rps_values, ces_values, 
	carbon_tax_values, cap_trade_clusters, cap_trade_values, 
	itc_values, ptc_values)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-22
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1905917 rows, 736956 columns and 5491849 nonzeros
Model fingerprint: 0x6af9b4fa
Coefficient statistics:
  Matrix range     [1e-03, 4e+00]
  Objective range  [1e-01, 5e+05]
  Bounds range     [2e+02, 7e+05]
  RHS range        [3e+00, 5e+04]
Presolve removed 384173 rows and 112827 columns
Presolve time: 4.43s
Presolved: 586036 rows, 1615998 columns, 5127436 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 1.27s

Barrier statistics:
 Free vars  : 20884
 AA' NZ     : 1.271e+07
 Factor NZ  : 7.012e+07 (roughly 1.5 GB of memory)
 Factor Ops : 1.147e+10 (less than 1 second per iteration)
 Threads    : 6

                  Object

(202×9 DataFrame
 Row │ ID     Resource                           Zone   Total_MW  Start_MW  Ch ⋯
     │ Int64  String                             Int64  Float64   Float64   Fl ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │     1  CA_N_batteries_1                       1   2653.8     2653.8     ⋯
   2 │     2  CA_N_biomass_1                         1    236.2      236.2
   3 │     3  CA_N_conventional_hydroelectric_1      1   7519.1     7519.1
   4 │     4  CA_N_geothermal_1                      1    987.4      987.4
   5 │     5  CA_N_hydroelectric_pumped_storag…      1      0.0     2276.9     ⋯
   6 │     6  CA_N_natural_gas_fired_combined_…      1   8885.03   11409.7
   7 │     7  CA_N_natural_gas_fired_combustio…      1      0.0     4732.2
   8 │     8  CA_N_natural_gas_steam_turbine_1       1      0.0        2.9
  ⋮  │   ⋮                    ⋮                    ⋮       ⋮         ⋮         ⋱
 196 │   196  WECC_WYCO_landbasedwind_class3_m…

In [31]:
print(no_policy_cost_results)
no_policy_tot_cost = no_policy_cost_results[1, :Objective]
print(no_policy_emissions_results)
print(dual_results)

1×10 DataFrame
 Row │ Fixed_Costs_Generation  Fixed_Costs_Storage  Fixed_Costs_Transmission  Variable_Costs  NSE_Costs  Objective  Real_System_Cost  Carbon_Tax  ITC      PTC     
     │ Float64                 Float64              Float64                   Float64         Float64    Float64    Float64           Float64     Float64  Float64 
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │                14332.6              194.778                   39.3913         9222.99    128.985    23918.8           23918.8         0.0      0.0      0.06×2 DataFrame
 Row │ Zone   Total_Emissions 
     │ Int64  Float64         
─────┼────────────────────────
   1 │     1        1.20692e7
   2 │     2        2.27673e7
   3 │     3        4.19586e7
   4 │     4        3.51458e7
   5 │     5        1.99765e6
   6 │     6        5.34976e76×4 DataFrame
 Row │ Zone   Cap_Trade  RPS   

### Test
The code below iteratively tests the optimization method for differing carbon policies. Although this is an extremely rough test, it should give an indication to whether or not the policies are generally working by showing the amount total system cost (which we expect to increase as we increase carbon standards) and by showing the total carbon output (which should decline)

In [49]:
# Lists of carbon policies
policy_to_test = Dict(
    "carbon_tax_values" => [50, 50, 50, 50, 50, 50],
    "ces_values" => [0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    "rps_values" => [0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
    "itc_values" => [0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
    "ptc_values" => [27.5, 27.5, 27.5, 27.5, 27.5, 27.5],
    "cap_trade_clusters" => [[1, 2, 3], [4, 5, 6]]
)
test_cap_trade_values = [10^6, 10^6]

for (key, value) in policy_to_test
    carbon_tax_values = key == "carbon_tax_values" ? value : [-1, -1, -1, -1, -1, -1]
    ces_values = key == "ces_values" ? value : [-1, -1, -1, -1, -1, -1]
    rps_values = key == "rps_values" ? value : [-1, -1, -1, -1, -1, -1]
    itc_values = key == "itc_values" ? value : [0, 0, 0, 0, 0, 0]
    ptc_values = key == "ptc_values" ? value : [0, 0, 0, 0, 0, 0]
    cap_trade_clusters = key == "cap_trade_clusters" ? value : []
    cap_trade_values = key == "cap_trade_clusters" ? test_cap_trade_values : []

    (generator_results,
    storage_results,
    transmission_results,
    nse_results,
    cost_results,
    time, emissions_results) = solve_cap_expansion(generators, G,
    demand, S, T, Z,
    nse,
    sample_weight, hours_per_period,
    lines, L,
    variability, Gurobi.Optimizer, rps_values, ces_values, 
	carbon_tax_values, cap_trade_clusters, cap_trade_values, 
	itc_values, ptc_values)
    println("Policy: ", key)
    println("Cost difference: ", cost_results[1, :Total_Costs] - no_policy_tot_cost)
    println(cost_results)
    println(emissions_results)

    # write results
    test_outpath = joinpath(outputs_directory, key)
    write_results(generator_results,
    storage_results,
    transmission_results,
    nse_results,
    cost_results,
    time,
    emissions_results,
    test_outpath)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-22
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1905919 rows, 736956 columns and 5615497 nonzeros
Model fingerprint: 0x7bdfa81c
Coefficient statistics:
  Matrix range     [1e-03, 1e+01]
  Objective range  [1e-01, 5e+05]
  Bounds range     [2e+02, 7e+05]
  RHS range        [3e+00, 1e+06]
Presolve removed 384173 rows and 112827 columns
Presolve time: 4.25s
Presolved: 586036 rows, 1616000 columns, 5235052 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 1.27s

Barrier statistics:
 Dense cols : 2
 Free vars  : 23731
 AA' NZ     : 1.282e+07
 Factor NZ  : 7.070e+07 (roughly 1.5 GB of memory)
 Factor Ops : 1.161e+10 (less than 1 second per iteration)
 Threads    : 6

        

## Test all Scenarios
Specific scenarios which allow for sensitivity testing and realistic scenarios are run in the test_scenarios function

In [36]:
include("test_scenarios.jl")
include("scenario_dicts.jl")

18-element Vector{Dict{String, Any}}:
 Dict("cap_trade_clusters" => [[1, 2], [5]], "name" => "aggressive", "cap_trade_values" => [25000000, 7000000], "ptc_values" => [27.5, 27.5, 27.5, 27.5, 27.5, 27.5], "carbon_tax_values" => [-1, -1, -1, -1, -1, -1], "itc_values" => [0.3, 0.3, 0.3, 0.3, 0.3, 0.3], "ces_values" => [0.9, 0.9, -1.0, 0.5, 0.6, 0.85], "rps_values" => [0.6, 0.6, 0.5, 0.15, 0.5, 0.3])
 Dict("cap_trade_clusters" => [[1, 2], [5]], "name" => "conservative", "cap_trade_values" => [4.0e7, 1.12e7], "ptc_values" => [27.5, 27.5, 0.0, 0.0, 27.5, 0.0], "carbon_tax_values" => [-1, -1, -1, -1, -1, -1], "itc_values" => [0.3, 0.3, 0.0, 0.0, 0.3, 0.0], "rps_values" => [0.6, 0.6, 0.5, 0.15, 0.5, 0.3], "ces_values" => [0.9, 0.9, -1.0, -1.0, 0.6, 0.8])
 Dict("cap_trade_clusters" => [[1, 2, 5]], "name" => "linked_cap_trade", "cap_trade_values" => [32000000], "ptc_values" => [27.5, 27.5, 27.5, 27.5, 27.5, 27.5], "carbon_tax_values" => [-1, -1, -1, -1, -1, -1], "itc_values" => [0.3, 0.3, 0.3, 0

In [37]:
test_scenarios(scenarios)

Modeling Scenario: aggressive
Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-22
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1905930 rows, 736956 columns and 6558985 nonzeros
Model fingerprint: 0xb16c9901
Coefficient statistics:
  Matrix range     [1e-03, 9e+00]
  Objective range  [1e-01, 3e+05]
  Bounds range     [2e+02, 7e+05]
  RHS range        [3e+00, 2e+07]
Presolve removed 384173 rows and 112827 columns
Presolve time: 4.95s
Presolved: 586036 rows, 1616011 columns, 6037968 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 1.41s

Barrier statistics:
 Dense cols : 13
 Free vars  : 23731
 AA' NZ     : 1.362e+07
 Factor NZ  : 7.155e+07 (roughly 1.5 GB of memory)
 Factor Ops : 1.184e+10 (less than 1 second per itera

Excessive output truncated after 539271 bytes.